In [ ]:
path = '/content/drive/MyDrive/book-recommender/'

In [ ]:
import pandas as pd
import numpy as np
import nltk
import string
import json
from nltk import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
df = pd.read_csv(path + 'book_preprocessed.csv')
df.shape

(4985, 14)

In [ ]:
def get_title(idx):
  return df.loc[idx, 'title']

In [ ]:
titles = df['title'].values.tolist()
len(titles)

4985

In [ ]:
en_stopwords = nltk.corpus.stopwords.words('english')
len(en_stopwords)

179

In [ ]:
def preprocessing_text(text):
  text = text.lower()
  text = word_tokenize(text)
  # remove english stop words
  text = [i for i in text if i not in en_stopwords]
  # remove punctuation
  text = [i for i in text if i not in string.punctuation]

  text = [i for i in text if i.isalnum()]
  return ' '.join(text)

In [ ]:
corpus = list(df['desc'].values)
len(corpus)

4985

In [ ]:
corpus = list(map(preprocessing_text, corpus))

In [ ]:
tfidf = TfidfVectorizer()
tf_transformed = tfidf.fit_transform(corpus)
tf_transformed.shape

(4985, 40191)

In [ ]:
cosine_sim = cosine_similarity(tf_transformed, tf_transformed)
cosine_sim.shape

(4985, 4985)

In [ ]:
cosine_sim_df = pd.DataFrame(cosine_sim)
cosine_sim_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,4945,4946,4947,4948,4949,4950,4951,4952,4953,4954,4955,4956,4957,4958,4959,4960,4961,4962,4963,4964,4965,4966,4967,4968,4969,4970,4971,4972,4973,4974,4975,4976,4977,4978,4979,4980,4981,4982,4983,4984
0,1.000000,0.009088,0.095090,0.198133,0.006923,0.013032,0.019479,0.067843,0.003698,0.011736,0.045791,0.016124,0.127813,0.007784,0.000000,0.015638,0.042835,0.036600,0.004824,0.002380,0.000000,0.000000,0.000000,0.003231,0.000000,0.004273,0.010178,0.000000,0.012860,0.007916,0.006495,0.019133,0.031153,0.025391,0.000000,0.023029,0.034853,0.020734,0.027136,0.034569,...,0.028834,0.012519,0.011881,0.000000,0.004011,0.020715,0.011020,0.016982,0.003318,0.000000,0.000000,0.004640,0.005915,0.004795,0.008120,0.0,0.011276,0.006190,0.009562,0.010173,0.010328,0.002484,0.018219,0.004959,0.022678,0.000000,0.025456,0.000000,0.026069,0.006078,0.025667,0.019341,0.0,0.013331,0.001967,0.000000,0.000000,0.015799,0.025825,0.000000
1,0.009088,1.000000,0.006704,0.002693,0.012139,0.006579,0.028629,0.006408,0.000000,0.000000,0.014029,0.010196,0.007690,0.016710,0.017704,0.000000,0.007242,0.000000,0.030131,0.031740,0.000000,0.018319,0.000000,0.027651,0.012037,0.007174,0.019881,0.007478,0.017533,0.006620,0.008266,0.002342,0.026805,0.035256,0.010603,0.000000,0.028745,0.013624,0.001146,0.012293,...,0.000000,0.011857,0.042781,0.000000,0.014377,0.020947,0.008564,0.007986,0.044296,0.013285,0.005688,0.003136,0.006910,0.007956,0.023620,0.0,0.025851,0.005530,0.005395,0.018488,0.022475,0.013562,0.010162,0.000795,0.015341,0.011009,0.014887,0.023122,0.003223,0.000000,0.007797,0.000000,0.0,0.017947,0.008641,0.007409,0.000000,0.034425,0.019218,0.012254
2,0.095090,0.006704,1.000000,0.386078,0.019127,0.000000,0.027962,0.005055,0.011476,0.017419,0.009438,0.038972,0.103196,0.044314,0.000000,0.000000,0.020375,0.000000,0.053803,0.046413,0.000000,0.021581,0.017928,0.064389,0.037771,0.003469,0.020586,0.000000,0.016689,0.005102,0.008594,0.003221,0.001660,0.009497,0.025292,0.000000,0.024297,0.019801,0.002290,0.052788,...,0.012120,0.019089,0.013156,0.000000,0.000000,0.008843,0.004459,0.008010,0.024399,0.007175,0.006478,0.011565,0.013841,0.000000,0.007339,0.0,0.010529,0.002326,0.005276,0.006424,0.003198,0.003715,0.010037,0.013189,0.003565,0.004463,0.003841,0.009127,0.006788,0.000000,0.005701,0.021981,0.0,0.011541,0.003203,0.006052,0.000000,0.001041,0.018971,0.022970
3,0.198133,0.002693,0.386078,1.000000,0.015077,0.006187,0.011435,0.010676,0.016600,0.003264,0.018714,0.035794,0.144862,0.069869,0.000000,0.000000,0.025615,0.000000,0.040799,0.038715,0.000000,0.012059,0.003646,0.037107,0.019683,0.012231,0.009231,0.030657,0.012776,0.012043,0.034121,0.004168,0.020237,0.007541,0.017181,0.003116,0.010667,0.002537,0.000000,0.059938,...,0.000000,0.003201,0.010521,0.006452,0.000000,0.002368,0.006778,0.031781,0.023103,0.004073,0.003341,0.004011,0.025056,0.005519,0.014199,0.0,0.012316,0.003148,0.000000,0.004863,0.005923,0.002244,0.026399,0.005444,0.006362,0.000000,0.008842,0.006198,0.012655,0.000000,0.021794,0.030086,0.0,0.000000,0.005379,0.004581,0.000000,0.007009,0.019584,0.000000
4,0.006923,0.012139,0.019127,0.015077,1.000000,0.019144,0.006710,0.012651,0.009798,0.009158,0.006410,0.007691,0.113624,0.019366,0.175818,0.243199,0.004362,0.036074,0.009334,0.003847,0.000000,0.000000,0.000000,0.012676,0.000000,0.009057,0.010128,0.011182,0.010472,0.013655,0.007120,0.015822,0.015610,0.003460,0.007111,0.073900,0.010663,0.000000,0.000000,0.008010,...,0.018964,0.014123,0.000000,0.000000,0.019373,0.008252,0.017808,0.002283,0.000000,0.000000,0.016741,0.000000,0.021069,0.045597,0.003424,0.0,0.010514,0.045579,0.006897,0.015178,0.015921,0.013012,0.008872,0.082121,0.024973,0.007066,0.000000,0.016222,0.016489,0.018775,0.006054,0.051241,0.0,0.015257,0.013745,0.005176,0.014549,0.012028,0.009613,0.007976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [ ]:
get_title(0), get_title(42), get_title(1657), get_title(2492), get_title(12)

('Harry Potter Collection',
 'Harry Potter and the Chamber of Secrets',
 'Harry Potter and History',
 'Only Time Will Tell',
 'Harry Potter Page to Screen: The Complete Filmmaking Journey')

In [ ]:
def get_recommend(book_idx: int):
  return dict(sorted(enumerate(cosine_sim[book_idx]), key=lambda x: x[1], reverse=True)[1:11])

In [ ]:
get_recommend(0)

{3: 0.19813335292707746,
 12: 0.12781276350174123,
 42: 0.341297612741696,
 262: 0.18114938509507672,
 747: 0.2023737868483844,
 1657: 0.21102461922508803,
 2492: 0.12583910068147341,
 3506: 0.14709436919136554,
 3778: 0.15871125772777367,
 4871: 0.15825452926577943}

In [ ]:
get_title(1)

'The Way of Kings, Part 2'

In [ ]:
get_title(43), get_title(46)

('Attack on Titan, Vol. 1', 'Last Stand of Dead Men')

In [ ]:
get_recommend(1)

{43: 0.12329977534148692,
 46: 0.0786174562831597,
 866: 0.09104701290681154,
 891: 0.1289488694072453,
 2427: 0.0996510825626173,
 2517: 0.10075133440197073,
 3090: 0.10751010156455473,
 3865: 0.08623707784416583,
 4502: 0.08619044166769282,
 4568: 0.13139394890537728}

In [ ]:
[i + 1 for i in get_recommend(3).keys()]

[3, 43, 1658, 3779, 263, 1, 2493, 4026, 2275, 2793]

In [ ]:
def get_list_recommend(idx):
  ret_data = dict(
    book_id=idx + 1,
    list_books=[i + 1 for i in get_recommend(idx).keys()]
  )
  return ret_data

In [ ]:
cosine_sim.shape[0]

4985

In [ ]:
%%time

desc_recommended = list(map(get_list_recommend, range(cosine_sim.shape[0])))

CPU times: user 14.1 s, sys: 36.9 ms, total: 14.1 s
Wall time: 14.1 s


In [ ]:
json.dump(desc_recommended, open('book_descriptions_recommend_data.json', 'w'), indent=4)